In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# Set parameters
CSV_PATH = os.getcwd()+"/csv_data/"

FRAME_SIZES = [ 64 , 72 ,80, 96 ,104,112,120]

IMG_HEIGHT = 96
IMG_WIDTH = 96

CSV_NAMES = os.listdir(CSV_PATH)

FEATURE_PATH = os.getcwd()+"/features_data/"
FRAME_PATH = os.getcwd()+"/frame_data/"
IMG_PATH = os.getcwd()+"/img_data/"

In [3]:
# Batches a given time series input
def _get_frames(_array,_frame_size,_stride,_re_size):

    # Total num of frames
    def _get_size(_array,_frame_size,_stride):
        _counter = 0
        for i in range(0,_array.shape[0]-_frame_size,_stride):
            _counter = _counter+1
        return _counter
    
    # Get the total num of frames
    _size = _get_size(_array,_frame_size,_stride)
    
    # Create an empty array
    _frame_array = np.zeros((_size,_re_size))

    _counter = 0
    for i in range(0,_array.shape[0]-_frame_size,_stride):
        # Get the frame
        _frame =  _array[i:i+_frame_size]
        _frame = _frame.reshape(_frame.shape+(1,1))
        _frame = tf.image.resize(_frame,(_re_size,1)).numpy()
        _frame = _frame.squeeze()
        
        # Normalize and assign the frame to the empty array
        _frame_array[_counter] = (1e-10+_frame-np.min(_frame))/(1e-10+np.max(_frame)-np.min(_frame))

        # Holds the index
        _counter = _counter +1
    return _frame_array


def get_all_frames(_csv_names,_csv_path,_frame_sizes,_stride = 0,_re_size =96):
    _all_frames = []
    for csv in _csv_names:
        try:
            # Get "close" prices of the candles
            df = pd.read_csv(_csv_path+csv)
            _close_arr = df["Close"].values

            for _frame_size in _frame_sizes:    
                # If _stride is not given , use the _frame_size as the _stride
                if _stride == 0:
                    _frame = _get_frames(_close_arr,_frame_size,_frame_size,_re_size)
                else:
                    _frame = _get_frames(_close_arr,_frame_size,_stride,_re_size)
                
                _all_frames.append(_frame)
        except:
            pass
    return _all_frames

In [4]:
frame_data = get_all_frames(CSV_NAMES,CSV_PATH,FRAME_SIZES,_stride = 4)

In [11]:
frame_data = np.array(frame_data)

<ipython-input-11-93c6bf553327>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  frame_data = np.array(frame_data)


In [14]:
frames=[]
for i in range(len(frame_data)):
    for j in range(frame_data[i].shape[0]):
        frames.append(frame_data[i][j])
frames = np.array(frames)

In [15]:
frames.shape

(1465342, 96)

In [26]:
np.save("frame_dataset.npy",frames)